In [9]:
import pandas as pd

from lib_all import *

Cấu hình mặc định

In [10]:
configs = {
    "num_classes": 21,
    "input_size": 300,
    "aspect_num": [4, 6, 6,6, 4, 4], #so bbox/ pixel  trong so cho cac sorce tu 1 -> 6
    "feature_maps": [38, 19, 10, 5, 3, 1],
    "steps": [8, 16, 32, 64, 100, 300], # size cua pixel
    "min_size": [30, 60, 111, 162, 213, 264],  #size cua default box
    "max_size": [60, 111, 162, 213, 264, 315], #size cua default box
    "aspect_ratio":[[2],[2,3],[2,3],[2,3],[2],[2]]
}

# Tạo ra các default box

In [11]:
class DefaultBox():
    def __init__(self, configs):
        self.input_size = configs["input_size"]
        self.feature_maps = configs["feature_maps"]
        self.steps = configs["steps"]
        self.min_size = configs["min_size"]
        self.max_size = configs["max_size"]
        self.aspect_ratio=configs["aspect_ratio"]

    def create_defaultbox(self):
        defaultbox_list = []
        for index, edge in enumerate(self.feature_maps):
            for x, y in itertools.product(range(edge), repeat=2):
                #fk: so luong pixel
                f_k = self.input_size/self.steps[index]

                center_x = (x+0.5)/ f_k
                center_y = (y+0.5)/ f_k

                #square box
                #small box
                s_k1 = self.min_size[index] / self.input_size
                defaultbox_list+=[center_x, center_y, s_k1, s_k1]

                #big box
                s_k2 =s_k1*(self.max_size[index]/self.input_size)
                s_k1_ = sqrt(s_k2)
                defaultbox_list+=[center_x, center_y, s_k1_, s_k1_]

                #rectangular
                for ar in self.aspect_ratio[index]:
                    defaultbox_list += [center_x, center_y, s_k1*sqrt(ar), s_k1/sqrt(ar)]
                    defaultbox_list += [center_x, center_y, s_k1/sqrt(ar), s_k1*sqrt(ar)]

        output =  torch.Tensor(defaultbox_list).view(-1, 4) # chuyen ve dang tensor
        output.clamp_(max =1, min=0) # can tren la 1, can duoi la 0

        return output

Cách hoạt động của create_defaultbox

In [12]:
for k, f in enumerate(configs["feature_maps"]):
    print("k la {}".format(k))
    print("f la {}".format(f))
    for i, j in itertools.product(range(f), repeat=2):
        print("i, j la {}, {}".format(i,j))

k la 0
f la 38
i, j la 0, 0
i, j la 0, 1
i, j la 0, 2
i, j la 0, 3
i, j la 0, 4
i, j la 0, 5
i, j la 0, 6
i, j la 0, 7
i, j la 0, 8
i, j la 0, 9
i, j la 0, 10
i, j la 0, 11
i, j la 0, 12
i, j la 0, 13
i, j la 0, 14
i, j la 0, 15
i, j la 0, 16
i, j la 0, 17
i, j la 0, 18
i, j la 0, 19
i, j la 0, 20
i, j la 0, 21
i, j la 0, 22
i, j la 0, 23
i, j la 0, 24
i, j la 0, 25
i, j la 0, 26
i, j la 0, 27
i, j la 0, 28
i, j la 0, 29
i, j la 0, 30
i, j la 0, 31
i, j la 0, 32
i, j la 0, 33
i, j la 0, 34
i, j la 0, 35
i, j la 0, 36
i, j la 0, 37
i, j la 1, 0
i, j la 1, 1
i, j la 1, 2
i, j la 1, 3
i, j la 1, 4
i, j la 1, 5
i, j la 1, 6
i, j la 1, 7
i, j la 1, 8
i, j la 1, 9
i, j la 1, 10
i, j la 1, 11
i, j la 1, 12
i, j la 1, 13
i, j la 1, 14
i, j la 1, 15
i, j la 1, 16
i, j la 1, 17
i, j la 1, 18
i, j la 1, 19
i, j la 1, 20
i, j la 1, 21
i, j la 1, 22
i, j la 1, 23
i, j la 1, 24
i, j la 1, 25
i, j la 1, 26
i, j la 1, 27
i, j la 1, 28
i, j la 1, 29
i, j la 1, 30
i, j la 1, 31
i, j la 1, 32
i, j la 1, 

# Test

In [13]:
def_box = DefaultBox(configs)
def_box_list =def_box.create_defaultbox()
print(pd.DataFrame(def_box_list.numpy()))

             0         1         2         3
0     0.013333  0.013333  0.100000  0.100000
1     0.013333  0.013333  0.141421  0.141421
2     0.013333  0.013333  0.141421  0.070711
3     0.013333  0.013333  0.070711  0.141421
4     0.013333  0.040000  0.100000  0.100000
...        ...       ...       ...       ...
8727  0.833333  0.833333  0.502046  1.000000
8728  0.500000  0.500000  0.880000  0.880000
8729  0.500000  0.500000  0.961249  0.961249
8730  0.500000  0.500000  1.000000  0.622254
8731  0.500000  0.500000  0.622254  1.000000

[8732 rows x 4 columns]
